In [1]:
#pip install azure-functions azure-functions-durable pandas azure-data-tables azure-storage-blob azure-functions python-dotenv langchain-openai langchain pinecone-client pyarrow adlfs fsspec langchain_pinecone azure-storage-file-datalake -q

In [2]:
#pip freeze ->requirements.txt

In [3]:
from datetime import datetime
import os

from external.blob_storage import BlobStorage
from external.data_lake_pandas import DataLakePandas
from external.data_lake_storage import DataLakeStorage

from app.repositories.pandas_loader import DataLakeLoader
from app.repositories.blob_storage_reader import BlobStorageReader
from app.repositories.pandas_reader import DataLakeReader

from app.use_cases.data_prep_delta_curated import DataPreprocessingDeltaCurated
from app.use_cases.data_prep_delta_raw import DataPreprocessingDeltaRaw 
import logging
from dotenv import load_dotenv
load_dotenv()
import pandas as pd

In [4]:
DELTA_DAYS = os.environ.get('DELTA_DAYS')
DELTA_HOURS = os.environ.get('DELTA_HOURS')
SCHEDULE = os.environ.get('CRON_SCHEDULE')

In [5]:
container_name = "landing"
blob_storage = BlobStorage(container_name)
bs_reader = BlobStorageReader(blob_storage)
dl_pandas = DataLakePandas()
dl_loader = DataLakeLoader(dl_pandas)

In [6]:
dl_storage = DataLakeStorage("raw")
dl_reader = DataLakeReader(dl_pandas,dl_storage)

use_case_delta_raw = DataPreprocessingDeltaRaw(bs_reader, dl_loader, dl_reader)  

use_case_delta_curated = DataPreprocessingDeltaCurated(dl_reader, dl_loader)  

In [7]:
timestamp = datetime.now()
filter_date = timestamp.strftime('%Y-%m-%d')
try:
    container_source = "landing"
    container_sink = "raw"
    filename= "data_from_youtube"
    filter = None
    delta_day= int(DELTA_DAYS)
    delta_hour= int(DELTA_HOURS)
    df_raw = use_case_delta_raw.execute(container_source, 
                            container_sink,
                            filename,
                            delta_day=delta_day,
                            delta_hour= delta_hour
                            )
    logging.info('external to raw')
    print('external to raw')
    container_source = "raw"
    container_sink = "curated"
    filename= "data_from_youtube"
    filter = None
    df_curated = use_case_delta_curated.execute(container_source, 
                                                container_sink,
                                                filename,
                                                delta_day,
                                                delta_hour, 
                                                filter)
    logging.info('raw to curated')
    print('raw to curated')
except Exception as e:
    msn = f'Error {e} , ejecutando la etl'
    logging.info(msn)
    print(msn)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   chanel_name       5 non-null      object 
 1   chanel_id         5 non-null      object 
 2   chanel_url        5 non-null      object 
 3   video_id          5 non-null      object 
 4   title             5 non-null      object 
 5   url               5 non-null      object 
 6   keywords          5 non-null      object 
 7   publish_date      5 non-null      object 
 8   relativeDateText  5 non-null      object 
 9   total_length      5 non-null      int64  
 10  total_views       5 non-null      int64  
 11  video_rating      0 non-null      float64
 12  description       5 non-null      object 
 13  duration          5 non-null      int64  
 14  caption_text_es   4 non-null      object 
dtypes: float64(1), int64(3), object(11)
memory usage: 732.0+ bytes
[2024]
abfs://sadatalakeproyecto

c:\Users\felip\Desktop\2024\MAESTRIA\proyecto\az-function-durable\app\use_cases\data_prep_delta_curated.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean["caption_text_es"] = df_clean["clean_title"] + ". " + df_clean["caption_text_es"].fillna(' ')
c:\Users\felip\Desktop\2024\MAESTRIA\proyecto\az-function-durable\app\use_cases\data_prep_delta_curated.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['text']= df_clean["caption_text_es"].apply(self._limpiar_texto)


[2024]
abfs://sadatalakeproyecto.dfs.core.windows.net/curated/data_from_youtube/data_from_youtube_2024
raw to curated


In [8]:
df_curated.info()

<class 'pandas.core.frame.DataFrame'>
Index: 164 entries, 0 to 195
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   chanel_name       164 non-null    object
 1   video_id          164 non-null    object
 2   source            164 non-null    object
 3   publish_date      164 non-null    object
 4   duration          164 non-null    int64 
 5   last_update_date  164 non-null    object
 6   title             164 non-null    object
 7   text              164 non-null    object
 8   year              164 non-null    int64 
dtypes: int64(2), object(7)
memory usage: 12.8+ KB


In [9]:
df_to_resume = df_curated[df_curated['last_update_date']==max(df_curated['last_update_date'])]
df_to_resume.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 192 to 195
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   chanel_name       4 non-null      object
 1   video_id          4 non-null      object
 2   source            4 non-null      object
 3   publish_date      4 non-null      object
 4   duration          4 non-null      int64 
 5   last_update_date  4 non-null      object
 6   title             4 non-null      object
 7   text              4 non-null      object
 8   year              4 non-null      int64 
dtypes: int64(2), object(7)
memory usage: 320.0+ bytes


In [10]:
from external.lang_chain_manager import LangChainManager
from external.pinecone_vdb import PineconeVectorDB
from app.repositories.pinecone_vdb_loader import PinceconeVDBRepository

c:\Users\felip\Desktop\2024\MAESTRIA\proyecto\az-function-durable\.venv\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [11]:
langchain_manager = LangChainManager()
pinecone_vdb = PineconeVectorDB()
vdb_repository= PinceconeVDBRepository(pinecone_vdb,langchain_manager)

In [12]:
id_column="video_id"
chunk_size=250
chunk_overlap=0

In [13]:
#crear documentos 
docs=vdb_repository.create_documents_from_df(df_to_resume)
len(docs)

4

In [14]:
#dividir documentos
splitted_docs=vdb_repository.split_documents(docs,chunk_size,chunk_overlap)
len(splitted_docs)

49

In [15]:
#agregar ids y metadata de particion a documentos
preprocess_docs,ids = vdb_repository.create_ids_documents(splitted_docs,id_column=id_column)

In [16]:
preprocess_docs[0].page_content

'qué dijo la fed y el mapa de puntos para bajada de tasas y actualización de bolsas. saludos cordiales bienvenido a bolsas hoy muy buenos días a todos jueves 13 de junio 8:20 de la mañana vamos a en unos minutitos a hacer un análisis de situación de las bolsas hasta que abra wall street después de ese ruido de mercado que sufrimos en el día de ayer digo sufrimos verdad ruido de mercado veremos cómo reaccionan hoy los inversores ese sentimiento después de que ayer primeramente conociemiento ipc en estados unidos del mes de mayo una décima por debajo lo estuve comentando todo igual que la conferencia de prensa del señor pell de la reserva federal las proyecciones del fond bueno que dejan sin cambios también de momento y que según el nuevo mapa de puntos de bajada el dot plot pues una sola en marzo nos dijeron tres ahora ya se habla solo de una para este año y que se dejan otras para la el próximo año aquí lo estuve comentando a medida de que se iban produciendo las declaraciones pues en 

In [17]:
preprocess_docs[0].metadata

{'chanel_name': 'Bolsas hoy',
 'video_id': 'OyFjPpm87T4',
 'source': 'https://www.youtube.com/watch?v=OyFjPpm87T4',
 'publish_date': '2024-06-12',
 'duration': 795,
 'last_update_date': '2024-06-13',
 'title': ' qué dijo la fed y el mapa de puntos para bajada de tasas y actualización de bolsas',
 'year': 2024,
 'split_tokens': 250,
 'part': 0}

Chat CPT 3.5 tiene un límite de 4096 tokens
GPT-4 "normal" de ChatGPT Plus admite 8.192 tokens.

## Resumen por canal y resumen general

resumenes
https://python.langchain.com/docs/use_cases/summarization/

In [18]:
from external.open_ia_chat import OpenAIChatModel
from external.lang_chain_manager import LangChainManager
from app.repositories.chat_model import ChatModelRepository
from app.repositories.agent_manager import AgentManagerRepository
from app.use_cases.data_resume import SummaryDocuments
from app.repositories.table_data_loader import TableDataLoader
from external.azure_table import AzureTableStorage
from app.use_cases.data_map_reduce_resume import MapReduceSummaryDocuments

In [19]:
open_ia_model = OpenAIChatModel(temperature=0.1)
lang_cahin_manager = LangChainManager()
chat_repository = ChatModelRepository(open_ia_model,lang_cahin_manager)
agent_repository =AgentManagerRepository(open_ia_model,lang_cahin_manager)

azure_table = AzureTableStorage()
table_data_loader = TableDataLoader(azure_table)

In [20]:
open_ia_model.model

'gpt-35-proyecto1'

In [21]:
documents = preprocess_docs

In [22]:
values_list = list(df_to_resume['video_id'].unique())
values_list

['OyFjPpm87T4', 'unmSnP44yPk', '_KD855_mh7E', 'ewaYY2Xx3Ak']

In [23]:
tokens_by_docs =agent_repository.count_tokens(documents)
tokens_by_docs

12089

In [24]:
use_case_summary = SummaryDocuments(chat_repository,agent_repository,table_data_loader)

In [25]:
result = use_case_summary.execute(preprocess_docs, df_to_resume, table_name='VideoSummaryTable')

Agrupar documentos
tokens transcripciones: 12089
Crear resumen
tokens transcripciones OyFjPpm87T4: 2477
tokens transcripciones unmSnP44yPk: 2728
tokens transcripciones _KD855_mh7E: 3473
tokens transcripciones ewaYY2Xx3Ak: 3411
elementos a guardar 4
guardar resumen


In [26]:
print(result[0]['resumen'])

Introducción:
En este informe se analiza la conferencia de prensa de la Reserva Federal de Estados Unidos (Fed) y el mapa de puntos para la bajada de tasas. También se examina la reacción de los mercados financieros, especialmente en relación con las bolsas de valores.

Bullet Points:
- La conferencia de prensa de la Fed reveló que se espera una sola bajada de tasas para este año, en lugar de las tres que se habían mencionado anteriormente.
- La Fed señaló que la economía de Estados Unidos continúa expandiéndose a un ritmo sólido, pero se mantendrá vigilante para evaluar si las políticas restrictivas están teniendo efecto.
- A pesar de las perspectivas de inflación más cautelosas de la Fed, Wall Street alcanzó nuevos máximos históricos, impulsado en parte por el fuerte desempeño de Apple.
- El Nasdaq también experimentó un fuerte aumento, acercándose a los 20.423 puntos, mientras que el S&P 500 mostró resistencia en su rango actual.
- El oro experimentó una subida, pero los futuros ind

In [27]:
print(result[1]['resumen'])

Introducción:
El informe se centra en el análisis técnico de Bitcoin y su comportamiento en el mercado el día 13 de junio de 2024. Se destaca la falta de rompimiento de máximos y la presencia de máximos decrecientes en el precio de Bitcoin.

Puntos importantes:
- A pesar de una fuerte subida inicial y un mercado alcista en general, Bitcoin no logró romper su máximo anterior del 10 de junio.
- El precio de Bitcoin se encuentra en un lateral de precios desde principios de año y no ha logrado alcanzar los objetivos de mediano y largo plazo.
- Si Bitcoin no logra romper los máximos actuales, se espera una situación de máximos decrecientes.
- Se señala la presencia de trampas alcistas en la acción del precio, lo que indica una posible toma de beneficios por parte de las manos fuertes.
- Se destaca la importancia de vigilar la apertura de Wall Street y la aparición de catalizadores que puedan cambiar la situación actual.

Conclusión:
El análisis técnico muestra que Bitcoin no ha logrado romp

## Map reduce

In [28]:
df =pd.DataFrame(result)
df.head()

,nombre_canal,numero_documentos,numero_tokens_iniciales,id_video,fecha_publicacion,fuente,titulo,RowKey,PartitionKey,resumen
0,Bolsas hoy,10,2477,OyFjPpm87T4,2024-06-12,https://www.youtube.com/watch?v=OyFjPpm87T4,qué dijo la fed y el mapa de puntos para baja...,2024-06-13_18_OyFjPpm87T4,OyFjPpm87T4,Introducción:\nEn este informe se analiza la c...
1,Bitcoin hoy,11,2728,unmSnP44yPk,2024-06-12,https://www.youtube.com/watch?v=unmSnP44yPk,qué hará bitcoin hoy 13/06/24 08:00 análisis ...,2024-06-13_18_unmSnP44yPk,unmSnP44yPk,Introducción:\nEl informe se centra en el anál...
2,Bolsas hoy,14,3473,_KD855_mh7E,2024-06-13,https://www.youtube.com/watch?v=_KD855_mh7E,evaluamos si bolsas seguirán a objetivos y com...,2024-06-13_18__KD855_mh7E,_KD855_mh7E,Introducción:\nEl texto analiza la situación d...
3,Bitcoin hoy,14,3411,ewaYY2Xx3Ak,2024-06-13,https://www.youtube.com/watch?v=ewaYY2Xx3Ak,"no se invalidad el escenario correctivo , zona...",2024-06-13_18_ewaYY2Xx3Ak,ewaYY2Xx3Ak,Introducción:\nEl informe se centra en el anál...


## prueba de caso de uso map reduce

In [29]:
use_case_map_reduce_summary = MapReduceSummaryDocuments(chat_repository,agent_repository,table_data_loader)

In [30]:
final_result = use_case_map_reduce_summary.execute( result, table_name='GeneralSummaryTable')

Crear resumen
tokens transcripciones: 1643
guardar resumen general


In [33]:
print(final_result['resumen'])

Introducción:
En este informe final se analizan cuatro temas diferentes relacionados con el trading y el análisis de mercado. Se examina la conferencia de prensa de la Reserva Federal de Estados Unidos (Fed) y el mapa de puntos para la bajada de tasas, el comportamiento de Bitcoin en el mercado, la situación de las bolsas y las acciones, y el análisis del mercado de Bitcoin. A continuación, se presentan los puntos más importantes de cada tema y se concluye con una breve conclusión para cada uno.

Conferencia de prensa de la Fed y mapa de puntos para la bajada de tasas:
- Se espera una sola bajada de tasas para este año, en lugar de las tres mencionadas anteriormente.
- La economía de Estados Unidos continúa expandiéndose a un ritmo sólido, pero se mantendrá vigilante para evaluar el impacto de las políticas restrictivas.
- Wall Street alcanzó nuevos máximos históricos, impulsado por el fuerte desempeño de Apple.
- El Nasdaq experimentó un fuerte aumento, acercándose a los 20.423 puntos